# Loading the `Moral-Stories` dataset
***
The dataset and code can be found <a href="https://github.com/demelin/moral_stories">here</a>.\
The authors provide 12k unique norms and, for some reason, additional 700k variations of the same norms, just with NaN fields every now and then. Zero additional information, but maybe I am overlooking something here?
* Might be for different tasks? But then they only provide a single label which is always 1 for any NaN rows...

In [14]:
# Data loaders returning `pandas` objects
import pandas as pd
import json

def get_moral_stories():
    data = pd.read_csv("data/moral_stories_datasets.tar.xz", compression="xz",
                       sep="\t", converters={0:json.loads})
    data = pd.DataFrame(list(data["moral_stories_datasets/"]))
    # filter out the 700k superfluous NaN entries
    data = data[data["label"].isna()]
    data = data.drop(["label"], axis=1)
    return data

def make_action_classification_dataframe(dataframe):
    '''
    Returns a dataframe with columns:
    `norm`, `action`, `situation`, `intention`, `consequence`,
    and `label`.
    The following columns differ from the original definition
    as returned by `get_moral_stories`:
        `action`: Is either the `moral_action` or `immoral_action`
        `label`: 1 if `action` is moral, 0 else
        `consequence`: The corresponding consequence of an action
    '''
    cols = [("moral_action", "moral_consequence"),
           ("immoral_action", "immoral_consequence")]
    immoral_df = dataframe.drop(["moral_action", "moral_consequence"], axis=1)
    moral_df = dataframe.drop(["immoral_action", "immoral_consequence"], axis=1)
    # rename columns
    moral_df.rename(columns={"moral_action":"action",
                            "moral_consequence":"consequence"},
                    inplace=True)
    immoral_df.rename(columns={"immoral_action":"action",
                            "immoral_consequence":"consequence"},
                    inplace=True)
    # add labels
    immoral_df["labels"] = 0
    moral_df["labels"] = 1
    
    data = pd.concat([moral_df, immoral_df], ignore_index=True)
    return data

# Sample task: Action classification
***
For starters, let's reproduce a task from the paper:
* Given an action, predict whether it is moral or immoral.
* For simplicity, we do not use the splits introduced in the paper, but rather random splitting

We start by loading the data as a `pandas.DataFrame`:

In [15]:
dataframe = get_moral_stories()
action_dataframe = make_action_classification_dataframe(dataframe)

## Task 1: Action only
***
We'll only give single sentences to the model for now. Let's start by feeding the actions.

In [16]:
def tokenize_and_split(dataset, tokenizer, sentence_col="text"):
    '''
    Takes a `datasets.Dataset` with train and test splits
    and applies the given tokenizer.
    Returns tokenized train and test split datasets
    '''
    def tokenize_function(examples):
        return tokenizer(examples[sentence_col], padding="max_length", truncation=True)

    tokenized_dataset = dataset.map(tokenize_function, batched=True)
    train_dataset = tokenized_dataset["train"]
    eval_dataset = tokenized_dataset["test"]
    return train_dataset, eval_dataset

from datasets import load_metric
import numpy as np
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [17]:
import datasets
test_split = 0.2
batch_size = 8


action_dataframe["task_input"] = action_dataframe.drop(["ID", "labels"], axis=1).agg(". ".join, axis=1)
dataset = datasets.Dataset.from_pandas(action_dataframe)
dataset = dataset.train_test_split(test_size=test_split)


In [18]:
from transformers import (
    AutoModelForSequenceClassification, DistilBertTokenizerFast,
     Trainer, TrainingArguments, AutoModelWithLMHead, AutoTokenizer,
)
import torch

model = "distilbert-base-uncased"
model = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelWithLMHead.from_pretrained(model)

c:\users\nikla\workspace\moral-stories-notebooks\env\lib\site-packages\transformers\models\auto\modeling_auto.py:843: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [21]:
prompt = "Today the weather is really nice and I am planning on "
inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

prompt_length = len(tokenizer.decode(inputs[0], skip_special_tokens=True, clean_up_tokenization_spaces=True))
outputs = model.generate(inputs, max_length=250, do_sample=False, top_p=0.95, top_k=60,
                        return_dict_in_generate=True, output_attentions=False,
                        output_hidden_states=True, output_scores=True)
#generated = prompt + tokenizer.decode(outputs[0])[prompt_length:]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [39]:
p = torch.softmax(outputs.scores[0], dim=1)

In [41]:
p.max()

tensor(0.5038)

In [ ]:
train_data, test_data = tokenize_and_split(dataset, tokenizer, "task_input")

In [ ]:
small_train_dataset = train_data.shuffle(seed=42).select(range(1000))
small_eval_dataset = test_data.shuffle(seed=42).select(range(1000))

In [ ]:
small_eval_dataset

In [ ]:
import torch
small_train_dataset.set_format("torch", columns=["input_ids","attention_mask","labels"])
dataloader = torch.utils.data.DataLoader(small_train_dataset, batch_size=2)

In [ ]:
inputs = next(iter(dataloader))
out = model(**inputs)

In [ ]:
a

In [ ]:
training_args = TrainingArguments(
    output_dir="results/",
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=50,                # how often to log
    save_steps=1000,
    save_total_limit=1,
    evaluation_strategy="epoch",     # when to run evaluation
)

In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_data,   # training dataset
    eval_dataset=test_data,     # evaluation dataset
    compute_metrics=compute_metrics,     # code to run accuracy metric
)
trainer.train()

In [ ]:
test_data.